In [2]:
import json
from pprint import pprint
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

In [3]:
file1_content = read_file('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/DEPMAP/rawdata/23Q4/README.txt')
file2_content = read_file('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/DEPMAP/rawdata/23Q2/README.txt')


In [4]:
c1 = "- Columns: genes (HGNC symbol and Ensembl ID)"
c2 = "Columns:\n"
# check if c1 contains "Columns: "
if c1.find("Columns: "):
    print("Found")

#
if c2.find("Columns: "):
    print("Found")

Found
Found


In [5]:
# create global variables for column and row subsections
column_subsection = False
row_subsection = False

def process_description(line_clean, subsection_dict):
    global column_subsection
    global row_subsection
    # if there is a "Rows" in the line, set row_subsection to True

    if "Rows: " in line_clean:
        row_subsection = True
        rows = line_clean.split("Rows: ")[1]
        subsection_dict["Rows"].append(rows)
        if "Columns: " in line_clean:
            column_subsection = True
            # line_clean would look like '- Columns: Gene - Rows: ScreenID' 
            # only want to append "Gene" to Columns
            columns = line_clean.split("Columns: ")[1].split(" - Rows: ")[0]
            subsection_dict["Columns"].append(columns)
        
    elif line_clean.startswith("Columns: "):
        columns = line_clean.split("Columns: ")[1].split(", ")
        subsection_dict["Columns"] += [col.strip() for col in columns]
    # else:
    #     subsection_dict["Description"] += (line_clean + " ")



def parse_content(content):
    sections = ["Overview", "Pipelines", "Files"]
    dict_content = {section: "" if section == "Overview" else {} for section in sections}
    
    content_lines = content.split("\n")
    section = None
    subsection = None
    
    
    
    for line in content_lines:
        # Skip empty lines
        if not line.strip():
            continue

        line_clean = line.strip("# ")
        # If line is a section heading
        if line_clean in sections:
            section = line_clean
            continue
        
        if section is None:
            continue    
        # If line is a subsection heading
        # Skip if we haven't reached a main section
        if "###" in line:
            
            subsection = line_clean
            # set global variables to False
            global column_subsection
            global row_subsection
            column_subsection = False
            row_subsection = False
            
            if section == "Files":
                dict_content[section][subsection] = \
                    {"Pipeline": "", "Description": "", "Columns": [], "Rows": []}
            elif section == "Pipelines":
                dict_content[section][subsection] = {"Description": ""}
            continue
        
        # For "Overview" section, content is combined into one string
        if section == "Overview":
            dict_content[section] += line_clean
            continue
            
        # For "Pipelines" and "Files" sections, content is added under specific subsection

        if section == "Files" and line_clean.startswith("Pipeline"):
            pipeline = line_clean.split("Pipeline: ")[1]
            dict_content[section][subsection]["Pipeline"] += pipeline
            continue
        
        # Below code only runs if we are in the "Files" section
        if "Columns:" in line_clean:
            column_subsection = True
            process_description(line_clean, dict_content[section][subsection])
        elif "Rows:" in line_clean:
            row_subsection = True
            process_description(line_clean, dict_content[section][subsection])
        elif column_subsection:
            dict_content[section][subsection]["Columns"] += [col.strip() for col in line_clean.strip("-").split("\n")]
        elif row_subsection:
            dict_content[section][subsection]["Rows"] += [row for row in line_clean.strip("-")]
        else:
            dict_content[section][subsection]["Description"] += (line_clean + " ")
                        

    return dict_content



In [6]:
output= parse_content(file1_content)
pprint(output["Overview"])
pprint(output["Pipelines"]["Expression"])
pprint(output["Files"]['OmicsExpressionTranscriptsTPMLogp1Profile.csv'])
pprint(output["Files"]['ScreenGeneEffect.csv'])

# for file in ['OmicsExpressionTranscriptsTPMLogp1Profile.csv', 'Model.csv

('This DepMap release contains data from CRISPR knockout screens from project '
 'Achilles, as well as genomic characterization data from the CCLE project.')
{'Description': 'DepMap expression data is quantified from RNAseq files using '
                'the GTEx pipelines. A detailed description of the pipelines '
                'and tool versions can be found here: '
                'https://github.com/broadinstitute/ccle_processing#rnaseq. We '
                'provide a subset of the data files outputted from this '
                'pipeline available on FireCloud. These are aligned to hg38. '}
{'Columns': [],
 'Description': 'RNAseq transcript tpm data using RSEM. Log2 transformed, '
                'using a pseudo-count of 1. ',
 'Pipeline': 'Expression',
 'Rows': ['Omics Profile IDs']}
{'Columns': ['Gene'],
 'Description': '_Post-Chronos_ Gene effect estimates for all screens Chronos '
                'processed by library, copy number corrected, scaled, screen '
              

In [7]:
from pprint import pprint 
omics = [_ for _ in output['Files'].keys() if _.startswith('Omics')]
pprint(omics)

['OmicsExpressionGenesExpectedCountProfile.csv',
 'OmicsExpressionProteinCodingGenesTPMLogp1.csv',
 'OmicsExpressionTranscriptsExpectedCountProfile.csv',
 'OmicsExpressionTranscriptsTPMLogp1Profile.csv',
 'OmicsExpressionAllGenesTPMLogp1Profile.csv',
 'OmicsExpressionAllGenesEffectiveLengthProfile.csv',
 'OmicsCNSegmentsProfile.csv',
 'OmicsCNGene.csv',
 'OmicsFusionFiltered.csv',
 'OmicsFusionUnfilteredProfile.csv',
 'OmicsSomaticMutations.csv',
 'OmicsSomaticMutationsProfile.csv',
 'OmicsSomaticMutationsMAFProfile.maf',
 'OmicsSomaticMutationsMatrixHotspot.csv',
 'OmicsSomaticMutationsMatrixDamaging.csv',
 'OmicsExpressionGeneSetEnrichment.csv',
 'OmicsExpressionGeneSetEnrichmentProfile.csv',
 'OmicsProfiles.csv',
 'OmicsDefaultModelProfiles.csv',
 'OmicsDefaultModelConditionProfiles.csv',
 'OmicsGuideMutationsBinaryKY.csv',
 'OmicsGuideMutationsBinaryHumagne.csv',
 'OmicsGuideMutationsBinaryAvana.csv']


In [8]:
omics_dicts = [
    {_ :output['Files'][_] } for _ in omics]

In [9]:
pprint(omics_dicts[0:4], width = 300)

[{'OmicsExpressionGenesExpectedCountProfile.csv': {'Columns': [], 'Description': 'RNAseq read count data from RSEM. ', 'Pipeline': 'Expression', 'Rows': ['Omics Profile IDs']}},
 {'OmicsExpressionProteinCodingGenesTPMLogp1.csv': {'Columns': [],
                                                    'Description': 'Gene expression TPM values of the protein coding genes for DepMap cell lines. Values are inferred from RNA-seq data using the RSEM tool and are reported after log2 transformation, using a pseudo-count of 1; log2(TPM+1). Additional '
                                                                   'RNA-seq-based expression measurements are available for download as part of the full DepMap Data Release More information on the DepMap Omics processing pipeline is available at <https://github.com/broadinstitute/depmap_omics>. ',
                                                    'Pipeline': 'Expression',
                                                    'Rows': []}},
 {'OmicsExp

In [11]:
import pandas as pd

# convert list of dictionaries to dataframe


AttributeError: 'list' object has no attribute 'items'